## Inputs

In [1]:
laps = 1
wear = 20

wearClearTrackRisk = 20

weather = 'Dry' 
#weather = 'Wet'
session = 'Q1'
#session = 'Q2'
#session = 'Race'

In [2]:
import re
import mechanize
from lxml import html
import os
import csv
from collections import defaultdict
import pandas as pd
from pathlib import Path # For data paths
import numpy as np
# Import external data
from calcs_edited import profileCalc, setupCalc, strategyCalc, wearCalc
from funcs import *
dicdata = defaultdict(list)

dataPath = str(Path.home()) + "/Documents/GitHub/GAPP/tempfiles"
filename = dataPath + "/data.dat"
file = open(filename, "r")
credentialCheck = int(float(file.readline()))
username = file.readline().strip()
password = file.readline().strip()

# Logon to GPRO using the logon information provided and store that under our session
browser = mechanize.Browser()
browser.open("https://gpro.net/gb/Login.asp")
browser.select_form(id="Form1")
browser.form["textLogin"] = username
browser.form["textPassword"] = password
browser.submit()

<response_seek_wrapper at 0x7fca0a3ddfa0 whose wrapped object = <closeable_response at 0x7fca0a3f5550 whose fp = <_io.BufferedReader name=61>>>

## Strategy Calculator

In [3]:
lapsUpper=(laps + 1)
print("Starting Strategy calculation")
strategy = strategyCalc(username, password, wear, laps)
stops, stintlaps, fuels, pitTimes, TCDs, FLDs, pitTotals, totals = [], [], [], [], [], [], [], []
for i in range(5):
    stops.append(strategy[0][i])
    stintlaps.append(strategy[1][i])
    fuels.append(strategy[2][i])
    pitTimes.append(strategy[3][i])
    TCDs.append(strategy[4][i])
    FLDs.append(strategy[5][i])
    pitTotals.append(strategy[6][i])
    totals.append(strategy[7][i])
lapsFuelLoadLower=(strategy[8][0])
lapsFuelLoadUpper=(strategy[8][1])

print("Getting track information")
GPROnextTrackName=(strategy[10])
GPROnextTrackLaps=(strategy[11])
GPROnextTrackLapDistance=(strategy[12])
GPROnextTrackDistance=(strategy[13])
GPROnextTrackPitInOut=(strategy[14])

trackList = [GPROnextTrackName, GPROnextTrackLaps, GPROnextTrackLapDistance, GPROnextTrackDistance, GPROnextTrackPitInOut]


grid = [stops, stintlaps, fuels, pitTimes, TCDs, FLDs, pitTotals, totals]
col=["Stops", "Stint Laps", "Fuel Load (L)", "Pit Time (s)", "TC Loss (s)", "Fuel Loss (s)", "Pit Total (s)", "Total (s)"]
row=["Extra Soft", "Soft", "Medium", "Hard", "Rain"]
df_strat = pd.DataFrame(grid).T
df_strat=df_strat.astype(float)
df_strat.columns = col
df_strat.index = row
#df_strat=df_strat.style.highlight_min(color = 'lightgreen', axis = 0, subset='Total (s)')
df_strat=df_strat.style.apply(lambda x: ['background: green' if x.name in [row[strategy[9]]] 
                              else '' for i in x], axis=1)
df_strat.data = df_strat.data.astype(int)
df_strat

Starting Strategy calculation
Getting track information


,Stops,Stint Laps,Fuel Load (L),Pit Time (s),TC Loss (s),Fuel Loss (s),Pit Total (s),Total (s)
Extra Soft,4,11,50,25,0,38,161,199
Soft,3,14,62,25,16,47,122,186
Medium,2,18,82,26,33,63,82,180
Hard,1,27,123,27,50,95,42,189
Rain,1,27,76,26,0,58,26,84


In [4]:
dftrack=pd.DataFrame(trackList)
track = ["Track Name", "Track Laps", "Track Length", "Track Distance", "PitInOut"]
dftrack.index = track
dftrack.columns = ['Info']

## Wear Calculator

In [5]:
# Request the car information page and scrape the car character and part level and wear data
browser.follow_link(url_regex=re.compile("UpdateCar"))
tree = html.fromstring(browser.response().get_data())
browser.back()

print('Getting wear info')
profilePower=(tree.xpath("normalize-space(//td[contains(text(), 'Power')]/../../tr[7]/td[1]/text())"))
profileHandling=(tree.xpath("normalize-space(//td[contains(text(), 'Power')]/../../tr[7]/td[2]/text())"))
profileAcceleration=(tree.xpath("normalize-space(//td[contains(text(), 'Power')]/../../tr[7]/td[3]/text())"))
wearlevelChassis=(int(tree.xpath("normalize-space(//b[contains(text(), 'Chassis')]/../../td[2]/text())")))
wearlevelEngine=(int(tree.xpath("normalize-space(//b[contains(text(), 'Engine')]/../../td[2]/text())")))
wearlevelFWing=(int(tree.xpath("normalize-space(//b[contains(text(), 'Front wing')]/../../td[2]/text())")))
wearlevelRWing=(int(tree.xpath("normalize-space(//b[contains(text(), 'Rear wing')]/../../td[2]/text())")))
wearlevelUnderbody=(int(tree.xpath("normalize-space(//b[contains(text(), 'Underbody')]/../../td[2]/text())")))
wearlevelSidepods=(int(tree.xpath("normalize-space(//b[contains(text(), 'Sidepods')]/../../td[2]/text())")))
wearlevelCooling=(int(tree.xpath("normalize-space(//b[contains(text(), 'Cooling')]/../../td[2]/text())")))
wearlevelGearbox=(int(tree.xpath("normalize-space(//b[contains(text(), 'Gearbox')]/../../td[2]/text())")))
wearlevelBrakes=(int(tree.xpath("normalize-space(//b[contains(text(), 'Brakes')]/../../td[2]/text())")))
wearlevelSuspension=(int(tree.xpath("normalize-space(//b[contains(text(), 'Suspension')]/../../td[2]/text())")))
wearlevelElectronics=(int(tree.xpath("normalize-space(//b[contains(text(), 'Electronics')]/../../td[2]/text())")))

carWearChassis = str(tree.xpath("normalize-space(//b[contains(text(), 'Chassis')]/../../td[4]/text())"))
if (carWearChassis == ""):
    carWearChassis = str(tree.xpath("normalize-space(//b[contains(text(), 'Chassis')]/../../td[4]/font/text())"))
wearChassis=(int((re.findall(r"\d+", carWearChassis))[0]))

carWearEngine = str(tree.xpath("normalize-space(//b[contains(text(), 'Engine')]/../../td[4]/text())"))
if (carWearEngine == ""):
    carWearEngine = str(tree.xpath("normalize-space(//b[contains(text(), 'Engine')]/../../td[4]/font/text())"))
wearEngine=(int((re.findall(r"\d+", carWearEngine))[0]))

carWearFrontWing = str(tree.xpath("normalize-space(//b[contains(text(), 'Front wing')]/../../td[4]/text())"))
if (carWearFrontWing == ""):
    carWearFrontWing = str(tree.xpath("normalize-space(//b[contains(text(), 'Front wing')]/../../td[4]/font/text())"))
wearFWing=(int((re.findall(r"\d+", carWearFrontWing))[0]))

carWearRearWing = str(tree.xpath("normalize-space(//b[contains(text(), 'Rear wing')]/../../td[4]/text())"))
if (carWearRearWing == ""):
    carWearRearWing = str(tree.xpath("normalize-space(//b[contains(text(), 'Rear wing')]/../../td[4]/font/text())"))
wearRWing=(int((re.findall(r"\d+", carWearRearWing))[0]))

carWearUnderbody = str(tree.xpath("normalize-space(//b[contains(text(), 'Underbody')]/../../td[4]/text())"))
if (carWearUnderbody == ""):
    carWearUnderbody = str(tree.xpath("normalize-space(//b[contains(text(), 'Underbody')]/../../td[4]/font/text())"))
wearUnderbody=(int((re.findall(r"\d+", carWearUnderbody))[0]))

carWearSidepod = str(tree.xpath("normalize-space(//b[contains(text(), 'Sidepods')]/../../td[4]/text())"))
if (carWearSidepod == ""):
    carWearSidepod = str(tree.xpath("normalize-space(//b[contains(text(), 'Sidepods')]/../../td[4]/font/text())"))
wearSidepods=(int((re.findall(r"\d+", carWearSidepod))[0]))

carWearCooling = str(tree.xpath("normalize-space(//b[contains(text(), 'Cooling')]/../../td[4]/text())"))
if (carWearCooling == ""):
    carWearCooling = str(tree.xpath("normalize-space(//b[contains(text(), 'Cooling')]/../../td[4]/font/text())"))
wearCooling=(int((re.findall(r"\d+", carWearCooling))[0]))

carWearGears = str(tree.xpath("normalize-space(//b[contains(text(), 'Gearbox')]/../../td[4]/text())"))
if (carWearGears == ""):
    carWearGears = str(tree.xpath("normalize-space(//b[contains(text(), 'Gearbox')]/../../td[4]/font/text())"))
wearGearbox=(int((re.findall(r"\d+", carWearGears))[0]))

carWearBrakes = str(tree.xpath(r"normalize-space(//b[contains(text(), 'Brakes')]/../../td[4]/text())"))
if (carWearBrakes == ""):
    carWearBrakes = str(tree.xpath("normalize-space(//b[contains(text(), 'Brakes')]/../../td[4]/font/text())"))
wearBrakes=(int((re.findall(r"\d+", carWearBrakes))[0]))

carWearSuspension = str(tree.xpath("normalize-space(//b[contains(text(), 'Suspension')]/../../td[4]/text())"))
if (carWearSuspension == ""):
    carWearSuspension = str(tree.xpath("normalize-space(//b[contains(text(), 'Suspension')]/../../td[4]/font/text())"))
wearSuspension=(int((re.findall(r"\d+", carWearSuspension))[0]))

carWearElectronics = str(tree.xpath("normalize-space(//b[contains(text(), 'Electronics')]/../../td[4]/text())"))
if (carWearElectronics == ""):
    carWearElectronics = str(tree.xpath("normalize-space(//b[contains(text(), 'Electronics')]/../../td[4]/font/text())"))
wearElectronics=(int((re.findall(r"\d+", carWearElectronics))[0]))

startWears = [wearChassis, wearEngine, wearFWing, wearRWing, wearUnderbody, wearSidepods, wearCooling, wearGearbox,
              wearBrakes, wearSuspension, wearElectronics]
wearlevels = [wearlevelChassis, wearlevelEngine, wearlevelFWing, wearlevelRWing, wearlevelUnderbody, wearlevelSidepods,
              wearlevelCooling, wearlevelGearbox, wearlevelBrakes, wearlevelSuspension, wearlevelElectronics]

print("Getting Driver information")
browser.follow_link(url_regex=re.compile("DriverProfile"))
tree = html.fromstring(browser.response().get_data())
browser.back()
driverConcentration = int(tree.xpath("normalize-space(//td[contains(@id, 'Conc')]/text())"))
driverTalent = int(tree.xpath("normalize-space(//td[contains(@id, 'Talent')]/text())"))
driverExperience = int(tree.xpath("normalize-space(//td[contains(@id, 'Experience')]/text())"))
driverFactor = (0.998789138 ** driverConcentration) * (0.998751839 ** driverTalent) * (
                0.998707677 ** driverExperience)

# Get the track details
print("Getting track information")
browser.follow_link(url_regex=re.compile("TrackDetails"))
tree = html.fromstring(browser.response().get_data())
browser.back()
trackName = str(tree.xpath("normalize-space(//h1[contains(@class, 'block')]/text())"))
trackName = trackName.strip()

raceWears = []
endWears = []
print("Calculating and applying car wear")
for i in range(len(startWears)):
    raceWears.append(round(float(wearCalc(startWears[i], int(wearlevels[i]), driverFactor, trackName,
                             wearClearTrackRisk, i)), 2))
    endWears.append(int(round(raceWears[i] + round(startWears[i], 0), 0)))

df = pd.DataFrame(np.column_stack([wearlevels, startWears, raceWears, endWears]).T, columns=['Chassis', 'Engine', 'FWing', 'RWing', 'Underbody', 'Sidepods', 'Cooling', 'Gearbox',
              'Brakes', 'Suspension', 'Electronics'])
df=df.astype(int)
df=df.rename({0: 'Level', 1: 'Start Wear', 2:'Race Wear', 3: 'End Wear'}, axis='index')
df = df.style.apply(lambda x: ["background: red" if v > 95 else "" for v in x], axis = 1)


Getting wear info
Getting Driver information
Getting track information
Calculating and applying car wear


In [6]:
profileAcceleration

'20'

## Setup Calculator

In [7]:
print("Starting Setup calcuation")
setup, temperature = setupCalc(username, password, weather, session)

print("Applying calculated setup")
frontWing=(str(setup[0]))
rearWing=(str(setup[1]))
engine=(str(setup[2]))
brakes=(str(setup[3]))
gear=(str(setup[4]))
suspension=(str(setup[5]))

df_setup = pd.DataFrame(np.column_stack([setup]).T, columns=['FWing', 'RWing', 'Engine', 'Brakes', 'Gear', 'Suspension'])


Starting Setup calcuation
Applying calculated setup


In [8]:
df

,Chassis,Engine,FWing,RWing,Underbody,Sidepods,Cooling,Gearbox,Brakes,Suspension,Electronics
Level,1,2,1,1,1,1,1,2,2,1,1
Start Wear,55,47,66,64,56,57,64,0,0,64,39
Race Wear,16,26,21,27,22,14,13,30,23,30,14
End Wear,72,74,87,92,79,72,78,30,24,95,54


In [9]:
df_setup

,FWing,RWing,Engine,Brakes,Gear,Suspension
0,422,542,628,566,435,575


In [10]:
df_strat

,Stops,Stint Laps,Fuel Load (L),Pit Time (s),TC Loss (s),Fuel Loss (s),Pit Total (s),Total (s)
Extra Soft,4,11,50,25,0,38,161,199
Soft,3,14,62,25,16,47,122,186
Medium,2,18,82,26,33,63,82,180
Hard,1,27,123,27,50,95,42,189
Rain,1,27,76,26,0,58,26,84


In [11]:
dftrack


,Info
Track Name,Suzuka
Track Laps,53
Track Length,5.86 km
Track Distance,310.6km
PitInOut,15.0


In [12]:
print(lapsFuelLoadLower)

5 L


Temperature	Humidity	Power	Handling	Acceleration	Driver Overall	Driver Energy	Driver Concentration	Driver Talent	Driver Aggressiveness	Driver Experience	Driver Technical Insight	Driver Stamina	Driver Charisma	Driver Motivation	Driver Reputation	Driver Weight	Driver Age	Chassis Level	Chassis Wear	Engine Level	Engine Wear	Front Wing Level	Front Wing Wear	Rear Wing Level	Rear Wing Wear	Underbody Level	Underbody Wear	Sidepods Level	Sidepods Wear	Cooling Level	Cooling Wear	Gearbox Level	Gearbox Wear	Brakes Level	Brakes Wear	Suspension Level	Suspension Wear	Electronics Level	Electronics Wear	Track_Suzuka	Track_Jeddah	Q1/Q2 Tyre Compound_Extra Soft / Extra Soft	Q1/Q2 Tyre Compound_Hard / Hard	Q1/Q2 Tyre Compound_Medium / Medium	Q1/Q2 Tyre Compound_Rain / Rain	Q1/Q2 Tyre Compound_Rain / Soft	Q1/Q2 Tyre Compound_Soft / Soft	Division_Amateur	Division_Rookie

In [13]:
Data = [temperature]

In [14]:
Data

[28]

In [16]:
# Request race strategy pace and scrape the race weather data
browser.follow_link(url_regex=re.compile("RaceSetup"))
tree = html.fromstring(browser.response().get_data())
browser.back()

rTempRangeOne = str(tree.xpath("normalize-space(//td[contains(text(), 'Temp')]/../../tr[2]/td[1]/text())"))
rTempRangeTwo = str(tree.xpath("normalize-space(//td[contains(text(), 'Temp')]/../../tr[2]/td[2]/text())"))
rTempRangeThree = str(tree.xpath("normalize-space(//td[contains(text(), 'Temp')]/../../tr[4]/td[1]/text())"))
rTempRangeFour = str(tree.xpath("normalize-space(//td[contains(text(), 'Temp')]/../../tr[4]/td[2]/text())"))
rHumRangeOne = str(tree.xpath("normalize-space(//td[contains(text(), 'Humidity')]/../../tr[2]/td[1]/text())"))
rHumRangeTwo = str(tree.xpath("normalize-space(//td[contains(text(), 'Humidity')]/../../tr[2]/td[2]/text())"))
rHumRangeThree = str(tree.xpath("normalize-space(//td[contains(text(), 'Humidity')]/../../tr[4]/td[1]/text())"))
rHumRangeFour = str(tree.xpath("normalize-space(//td[contains(text(), 'Humidity')]/../../tr[4]/td[2]/text())"))